In [ ]:
# pip install langchain_community

In [ ]:
# pip install faiss-cpu

In [ ]:
# pip install langchain_google_genai

In [ ]:
# pip install unstructured

In [ ]:
import os
from dotenv import load_dotenv
from langchain_community.document_loaders import DirectoryLoader, UnstructuredHTMLLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_core.prompts import ChatPromptTemplate, PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough

load_dotenv()
GEMINI_API_KEY = "_____"

print("\n" + "="*70)
print("🎓 LANGCHAIN RAG DEMO - Core Concepts")
print("="*70)

# ============================================================
# PART 1: Document Loading & Processing (LangChain Loaders)
# ============================================================
print("\n📚 PART 1: Document Loading")
print("-" * 70)

loader = DirectoryLoader(
    "coffee_pages",
    glob="**/*.html",
    loader_cls=UnstructuredHTMLLoader,
    show_progress=True
)
docs = loader.load()
print(f"✅ Loaded {len(docs)} documents using LangChain DirectoryLoader")

# ============================================================
# PART 2: Text Splitting (LangChain Text Splitters)
# ============================================================
print("\n✂️  PART 2: Text Splitting")
print("-" * 70)

splitter = RecursiveCharacterTextSplitter(
    chunk_size=800,
    chunk_overlap=100,
    separators=["\n\n", "\n", ".", " ", ""]
)
chunks = splitter.split_documents(docs)
print(f"✅ Split into {len(chunks)} chunks using RecursiveCharacterTextSplitter")
print(f"   - Chunk size: 800 characters")
print(f"   - Overlap: 100 characters")

# ============================================================
# PART 3: Embeddings & Vector Store (LangChain Embeddings)
# ============================================================
print("\n🧮 PART 3: Embeddings & Vector Store")
print("-" * 70)

embeddings = GoogleGenerativeAIEmbeddings(
    model= 'models/gemini-embedding-001', # "models/embedding-001",
    google_api_key=GEMINI_API_KEY
)
print("✅ Initialized Gemini embeddings")

vectorstore = FAISS.from_documents(chunks, embeddings)
print("✅ Created FAISS vector store")


🎓 LANGCHAIN RAG DEMO - Core Concepts

📚 PART 1: Document Loading
----------------------------------------------------------------------


100%|██████████| 15/15 [00:00<00:00, 42.56it/s]


✅ Loaded 15 documents using LangChain DirectoryLoader

✂️  PART 2: Text Splitting
----------------------------------------------------------------------
✅ Split into 45 chunks using RecursiveCharacterTextSplitter
   - Chunk size: 800 characters
   - Overlap: 100 characters

🧮 PART 3: Embeddings & Vector Store
----------------------------------------------------------------------
✅ Initialized Gemini embeddings
✅ Created FAISS vector store


In [ ]:
# ============================================================
# PART 4: Basic Retriever (Core LangChain Concept)
# ============================================================
print("\n🔍 PART 4: Basic Retriever")
print("-" * 70)

# This is the FUNDAMENTAL LangChain retriever
retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}  # Return top 4 results
)


🔍 PART 4: Basic Retriever
----------------------------------------------------------------------


In [ ]:
def demo_basic_retrieval(query):
    print(f"\n{'='*70}")
    print(f"🔍 DEMO: Basic Retrieval")
    print(f"{'='*70}")
    print(f"Query: {query}\n")

    docs = retriever.invoke(query)
    print(f"Retrieved {len(docs)} documents:\n")

    for i, doc in enumerate(docs, 1):
        print(f"{i}. {doc.page_content[:150]}...\n")

In [ ]:
demo_basic_retrieval("What is Ashwagandha Coffee?")


🔍 DEMO: Basic Retrieval
Query: What is Ashwagandha Coffee?

Retrieved 4 documents:

1. Ashwagandha Coffee (Adaptogenic Latte)

Updated on August 17, 2025

Disclaimer: This page shares general culinary and cultural information. It is not ...

2. Mushroom & Ashwagandha Coffee (Adaptogen Blend)

Updated on August 17, 2025

Disclaimer: This page shares general culinary and cultural information. I...

3. Optional: froth and dust with cinnamon or cardamom.

Ashwagandha can taste earthy and slightly bitter; spices like cardamom smooth the edge. If the po...

4. ½–1 tsp instant coffee or a single espresso shot

¼–½ tsp ashwagandha powder (culinary grade)

¼ tsp cinnamon or cardamom (optional)

Sweetener to tas...



In [ ]:
# ============================================================
# PART 5: Prompt Templates + DEMO
# ============================================================
"""
NOTE: Prompt templates are reusable prompt structures:
- Define placeholders with {variable_name}
- Ensure consistent prompting
- Easy to modify and test

Types:
- PromptTemplate: Simple string templates
- ChatPromptTemplate: For chat models (system/user/assistant roles)
"""


'\nNOTE: Prompt templates are reusable prompt structures:\n- Define placeholders with {variable_name}\n- Ensure consistent prompting\n- Easy to modify and test\n\nTypes:\n- PromptTemplate: Simple string templates\n- ChatPromptTemplate: For chat models (system/user/assistant roles)\n'

In [ ]:
# Simple template
simple_template = PromptTemplate(
    template="Answer this question: {question}",
    input_variables=["question"]
)

# RAG template (includes context)
rag_template = ChatPromptTemplate.from_template("""
You are a coffee expert assistant. Use the context below to answer the question.
If the answer is not in the context, say "I don't have that information."

Context:
{context}

Question: {question}

Answer:""")

In [ ]:
# ============================================================
# PART 6: LLM Integration
# ============================================================

In [ ]:
llm = ChatGoogleGenerativeAI(
    model='gemini-2.5-flash',
    google_api_key = GEMINI_API_KEY,
    temperature = 0)

In [ ]:
# ============================================================
# PART 7: Chains + DEMOS
# ============================================================

"""
NOTE: Chains connect components using the | operator :
- Data flows left to right
- Each step transforms the input
- Modular and composable

Example: prompt | llm | parser
1. Prompt formats the input
2. LLM generates response
3. Parser extracts text
"""

In [ ]:
# Helper function
def format_docs(docs):
    """Convert list of documents to single string"""
    return "\n\n".join(doc.page_content for doc in docs)

In [ ]:
# Chain 1 :
simple_chain = simple_template | llm | StrOutputParser()

def demo_simple_chain(query) :
  response = simple_chain.invoke(query)
  print(response)

demo_simple_chain("What are the benefits of coffee?")

# Note: This uses LLM's general knowledge, not your documents!

Coffee, when consumed in moderation and without excessive sugar or cream, offers a surprising array of potential health benefits due to its rich content of antioxidants and other bioactive compounds, primarily caffeine.

Here are some of the key benefits:

1.  **Increased Energy Levels and Alertness:** Caffeine, the most well-known active ingredient in coffee, is a stimulant that blocks adenosine, a neurotransmitter that makes you feel tired. This leads to increased alertness, improved focus, and reduced fatigue.

2.  **Enhanced Physical Performance:** Caffeine can improve various aspects of physical performance, including endurance, strength, and power. It does this by stimulating the nervous system, increasing adrenaline levels, and mobilizing fatty acids from fat tissues, making them available as fuel.

3.  **Rich in Antioxidants:** Coffee is one of the largest sources of antioxidants in the Western diet. These antioxidants help fight free radicals in the body, reducing oxidative st

In [ ]:
# Chain 2 : RAG Template
rag_chain = (
    {
    "context" : retriever | format_docs, # Retrive & format_docs
    "question" : RunnablePassthrough() # PAss question through
}|rag_template | llm | StrOutputParser()
)

In [ ]:
def demo_rag_chain(query) :
  response = rag_chain.invoke(query)
  print(response)

demo_rag_chain("What is Ashwagandha coffee and its benefits?")
# grounded answer!

Ashwagandha coffee combines roasted coffee with powdered ashwagandha, an herb widely used in Indian traditions. The goal is to enjoy a familiar cup while layering in earthy, slightly bitter herbal notes. Home cooks typically treat this like a latte with spices for balance.

I don't have information on its benefits.


In [ ]:
demo_simple_chain("Who is the PM of India?")

The current Prime Minister of India is **Narendra Modi**.


In [ ]:
demo_rag_chain("Who is the PM of India?")

I don't have that information.


In [ ]:
# 1. Similariy
similarity_retriever = vectorstore.as_retriever(
    search_type="similarity",
    search_kwargs={"k": 4}
)

In [ ]:
# 2. MMR
mmr_retriever = vectorstore.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 4, "fetch_k": 10}
)

In [ ]:
# 3. Threshold
threshold_retriever = vectorstore.as_retriever(
    search_type="similarity_score_threshold",
    search_kwargs={"score_threshold": 0.5, "k": 4}
)

In [ ]:
# similarity_retriever.invoke("What is mushroom coffee?")

In [ ]:
# mmr_retriever.invoke("What is mushroom coffee?")

In [ ]:
# threshold_retriever.invoke("What is mushroom coffee?")

In [ ]:
print("\n Multi-Query Retrieval")
"""
NOTE: Multi-query improves retrieval by:
- Generating multiple phrasings of the same question
- Searching with each variation
- Combining results (deduplication)

Why it helps:
- User's phrasing might not match documents
- Captures synonyms and related concepts
- Better recall (finds more relevant docs)
"""


 Multi-Query Retrieval


"\nNOTE: Multi-query improves retrieval by:\n- Generating multiple phrasings of the same question\n- Searching with each variation\n- Combining results (deduplication)\n\nWhy it helps:\n- User's phrasing might not match documents\n- Captures synonyms and related concepts\n- Better recall (finds more relevant docs)\n"

In [ ]:
def generate_multi_queries(query, num_queries=3):

    multi_query_prompt = PromptTemplate(
        template="""Generate {num_queries} different versions of this question.
Output one per line, numbered.

Original: {question}

Variations:""",
        input_variables=["question", "num_queries"]
    )

    chain = multi_query_prompt | llm | StrOutputParser()
    response = chain.invoke({"question": query, "num_queries": num_queries})

    queries = [query]
    for line in response.split('\n'):
        cleaned = line.strip().lstrip('0123456789.)-• ').strip()
        if cleaned and len(cleaned) > 10:
            queries.append(cleaned)

    return queries[:num_queries + 1]

In [ ]:
def multi_query_retrieve(query):
    """Retrieve using multiple query variations"""

    queries = generate_multi_queries(query, num_queries=2)

    all_docs = []
    seen_content = set()

    for q in queries:
        docs = retriever.invoke(q)
        for doc in docs:
            if doc.page_content not in seen_content:
                seen_content.add(doc.page_content)
                all_docs.append(doc)

    return queries, all_docs


In [ ]:
def demo_multi_query(query):
  queries,doc = multi_query_retrieve(query)
  for i ,q in enumerate(queries,1):
    print(f"{i}. Query: {q}")
  single_docs = retriever.invoke(query)
  print("Results")
  print(f"   Single query: {len(single_docs)} docs")
  print(f"   Multi-query:  {len(docs)} unique docs")
  print(f"   Improvement:  +{len(docs) - len(single_docs)} docs")

  context = format_docs(docs)
  answers = llm.invoke(rag_template.format(context=context, question=query)).content

In [ ]:
demo_multi_query("What are the health benefits of Ashwagandha coffee?")

1. Query: What are the health benefits of Ashwagandha coffee?
2. Query: What positive health effects does Ashwagandha coffee provide?
3. Query: How does consuming Ashwagandha coffee contribute to overall well-being?
Results
   Single query: 4 docs
   Multi-query:  15 unique docs
   Improvement:  +11 docs


In [ ]:
# Query decomposition :

In [ ]:
# ============================================================
# PART 10: Query Decomposition + DEMO
# ============================================================
print("\n🧩 PART 10: Query Decomposition")
print("-" * 70)

# """
# NOTE: Query decomposition breaks complex questions:
# - Complex question → Multiple sub-questions
# - Answer each separately
# - Synthesize sub-answers into final answer

# When to use:
# - Multi-part questions ("Compare X and Y")
# - Questions requiring multiple facts
# - Complex analysis
# """


🧩 PART 10: Query Decomposition
----------------------------------------------------------------------


In [ ]:
def decompose_query(complex_query):
    """Break complex query into simpler sub-questions"""

    decomposition_prompt = PromptTemplate(
        template="""Break this into 2-3 simpler sub-questions.
Output one per line, numbered.

Complex: {question}

Sub-questions:""",
        input_variables=["question"]
    )

    chain = decomposition_prompt | llm | StrOutputParser()
    response = chain.invoke({"question": complex_query})

    sub_queries = []
    for line in response.split('\n'):
        cleaned = line.strip().lstrip('0123456789.)-• ').strip()
        if cleaned and len(cleaned) > 10:
            sub_queries.append(cleaned)
    return sub_queries

In [ ]:
def answer_with_decomposition(complex_query):
    """Answer complex query via decomposition"""
    # Step 1: Decompose
    sub_queries = decompose_query(complex_query)
    print(sub_queries)

    # Step 2: Answer each
    sub_answers = []
    for sq in sub_queries:
        answer = rag_chain.invoke(sq)
        sub_answers.append({"question": sq, "answer": answer})

    # Step 3: Synthesize
    synthesis_prompt = ChatPromptTemplate.from_template("""
Original: {original_question}

Sub-answers:
{sub_answers}

Provide comprehensive answer based on sub-answers:""")

    sub_text = "\n\n".join([
        f"Q: {sa['question']}\nA: {sa['answer']}"
        for sa in sub_answers
    ])

    synthesis_chain = synthesis_prompt | llm | StrOutputParser()
    final = synthesis_chain.invoke({
        "original_question": complex_query,
        "sub_answers": sub_text
    })

    return sub_queries, sub_answers, final

print("✅ Created query decomposition functions")

✅ Created query decomposition functions


In [ ]:
# answer_with_decomposition("Compare health benefits and side effects of Ashwagandha coffee vs regular coffee")


In [ ]:
def demo_query_decomposition(complex_query):
    """
    Demo: Query decomposition
    Shows how complex questions are broken down
    """
    print(f"\n{'='*70}")
    print(f"🧩 DEMO 6: Query Decomposition")
    print(f"{'='*70}")
    print(f"Complex Query: {complex_query}\n")

    sub_queries, sub_answers, final = answer_with_decomposition(complex_query)

    print("Step 1 - Decomposed:")
    for i, sq in enumerate(sub_queries, 1):
        print(f"{i}. {sq}")

    print("\nStep 2 - Sub-answers:")
    for i, sa in enumerate(sub_answers, 1):
        print(f"\n{i}. Q: {sa['question']}")
        print(f"   A: {sa['answer'][:100]}...")

    print(f"\nStep 3 - Final Answer:")
    print(f"🎯 {final[:250]}...\n")

In [ ]:
# RUN DEMO
demo_query_decomposition("Compare health benefits and side effects of Ashwagandha coffee vs regular coffee")



🧩 DEMO 6: Query Decomposition
Complex Query: Compare health benefits and side effects of Ashwagandha coffee vs regular coffee

['What are the health benefits and potential side effects of Ashwagandha coffee?', 'What are the health benefits and potential side effects of regular coffee?', 'What are the key differences in health benefits and side effects between Ashwagandha coffee and regular coffee?']
Step 1 - Decomposed:
1. What are the health benefits and potential side effects of Ashwagandha coffee?
2. What are the health benefits and potential side effects of regular coffee?
3. What are the key differences in health benefits and side effects between Ashwagandha coffee and regular coffee?

Step 2 - Sub-answers:

1. Q: What are the health benefits and potential side effects of Ashwagandha coffee?
   A: I don't have that information....

2. Q: What are the health benefits and potential side effects of regular coffee?
   A: I don't have that information....

3. Q: What are the key diffe

In [ ]:
# Conversation memory

In [ ]:
conversation_memory = {}

In [ ]:
def get_memory(session_id="default"):
    if session_id not in conversation_memory:
        conversation_memory[session_id] = {"history": []}
    return conversation_memory[session_id]

def add_user_message(memory, message):
    memory["history"].append({"role": "user", "content": message})

def add_assistant_message(memory, message):
    memory["history"].append({"role": "assistant", "content": message})

def get_history_text(memory):
    history = memory.get("history", [])
    if not history:
        return ""
    formatted = "\n\nPrevious conversation:\n"
    for msg in history:
        role = "User" if msg["role"] == "user" else "Assistant"
        formatted += f"{role}: {msg['content']}\n"
    return formatted

def clear(memory):
    memory["history"].clear()

def chatbot(session_id="chat"):
    """
    Interactive chatbot with conversational memory.
    Type 'exit' to stop or 'clear' to reset memory.
    """
    print(f"\n{'='*70}")
    print("🤖 Chatbot with Conversational Memory")
    print("Type 'exit' to quit or 'clear' to reset memory.")
    print(f"{'='*70}\n")

    memory = get_memory(session_id)
    clear(memory)

    while True:
        query = input("👤 You: ").strip()
        if not query:
            continue
        if query.lower() == "exit":
            print("👋 Goodbye!")
            break
        if query.lower() == "clear":
            clear(memory)
            print("🧹 Memory cleared.")
            continue

        # Step 1: Retrieve documents (your retriever logic)
        docs = retriever.invoke(query)
        context = format_docs(docs)

        # Step 2: Combine context + memory
        prompt = f"""You are a coffee expert. Use context and history.

Context:
{context}
{get_history_text(memory)}

Question: {query}

Answer:"""

        # Step 3: Generate response
        response = llm.invoke(prompt).content

        # Step 4: Print & store
        print(f"🤖 Bot: {response}\n")

        add_user_message(memory, query)
        add_assistant_message(memory, response)

# RUN CHATBOT
chatbot()


🤖 Chatbot with Conversational Memory
Type 'exit' to quit or 'clear' to reset memory.

👤 You: what is watermelon juice
🤖 Bot: As a coffee expert, my knowledge is focused on coffee preparations and related ingredients, as detailed in the provided context about Cardamom Coffee, Masala Coffee, Jaggery Coffee, and Ashwagandha Coffee.

The information provided does not contain any details about watermelon juice.

👤 You: how this should be drunk 
🤖 Bot: Based on the provided context, here's how these coffees, particularly cold brew and those with an Ayurvedic-inspired approach, should be drunk:

1.  **Cold Brew Specifics:**
    *   **Serve over ice:** Always serve the prepared cold brew concentrate over ice. Using large cube ice is recommended as it melts slower and preserves flavor.
    *   **Dilute to taste:** The concentrate is strong; dilute it 1:1 with water or milk, or adjust to your personal preference.
    *   **Sweeten or add spice syrup:** You can add sweetness or flavor with spice